# 🧬 POLYMORPH-LITE on Google Colab

This notebook runs the POLYMORPH-LITE Lab OS on Google Colab.

## Features
- FastAPI Backend with AI (PMM) service
- Real-time Raman spectroscopy simulation
- Polymorph detection and tracking
- Safety guardrails and gating

**Run each cell in order.**

## 1️⃣ Setup Environment

In [ ]:
# Clone the repository
!git clone https://github.com/dawsonblock/POLYMORPH_LITE_MAIN.git
%cd POLYMORPH_LITE_MAIN

# Check Python version
!python --version

In [ ]:
# Install core dependencies
!pip install -q fastapi uvicorn sqlalchemy asyncpg aiosqlite pydantic pydantic-settings
!pip install -q numpy scipy torch torchvision
!pip install -q PyJWT httpx redis aiohttp
!pip install -q prometheus-client structlog python-multipart
!pip install -q pyngrok  # For public URL

print("✅ Core dependencies installed")

In [ ]:
# Install remaining dependencies from requirements
!pip install -q -r requirements.txt 2>/dev/null || echo "Some optional deps may have failed - OK"

print("✅ Requirements installed")

## 2️⃣ Configure Environment Variables

In [ ]:
import os

# Set environment variables for Colab
os.environ['P4_ENVIRONMENT'] = 'development'
os.environ['P4_DATABASE_URL'] = 'sqlite+aiosqlite:///./polymorph.db'  # SQLite for Colab
os.environ['P4_REDIS_URL'] = ''  # Disable Redis in Colab
os.environ['P4_JWT_SECRET'] = 'colab_demo_secret_key_change_in_prod'
os.environ['P4_LOG_LEVEL'] = 'INFO'
os.environ['P4_DAQ_BACKEND'] = 'simulator'
os.environ['P4_RAMAN_PROVIDER'] = 'simulator'
os.environ['PMM_CHECKPOINT_DIR'] = '/content/POLYMORPH_LITE_MAIN/checkpoints'
os.environ['PMM_AUTO_LOAD'] = 'false'

# Create checkpoints directory
!mkdir -p /content/POLYMORPH_LITE_MAIN/checkpoints

print("✅ Environment configured")
print(f"   Database: {os.environ['P4_DATABASE_URL']}")
print(f"   Checkpoints: {os.environ['PMM_CHECKPOINT_DIR']}")

## 3️⃣ Initialize Database

In [ ]:
# Initialize SQLite database
import sys
sys.path.insert(0, '/content/POLYMORPH_LITE_MAIN')

import asyncio
from sqlalchemy.ext.asyncio import create_async_engine, AsyncSession
from sqlalchemy.orm import sessionmaker

# Import models to register them
from retrofitkit.db.base import Base
from retrofitkit.db import models

async def init_db():
    engine = create_async_engine(
        os.environ['P4_DATABASE_URL'],
        echo=False
    )
    async with engine.begin() as conn:
        await conn.run_sync(Base.metadata.create_all)
    print("✅ Database tables created")
    return engine

# Run in event loop
engine = asyncio.get_event_loop().run_until_complete(init_db())

## 4️⃣ Test Core Imports

In [ ]:
# Test that all core modules import correctly
try:
    from retrofitkit.core.gating import GatingEngine
    print("✓ Gating engine")
except Exception as e:
    print(f"✗ Gating: {e}")

try:
    from retrofitkit.core.safety.guardrails import SafetyGuardrails
    print("✓ Safety guardrails")
except Exception as e:
    print(f"✗ Safety: {e}")

try:
    from retrofitkit.api.metrics import router as metrics_router
    print("✓ Prometheus metrics")
except Exception as e:
    print(f"✗ Metrics: {e}")

try:
    from retrofitkit.api.middleware.org_context import OrgContextMiddleware
    print("✓ Org context middleware")
except Exception as e:
    print(f"✗ Middleware: {e}")

print("\n✅ Core modules ready")

## 5️⃣ Start Backend Server (with ngrok)

In [ ]:
# Get ngrok auth token (optional but recommended)
# Sign up at https://ngrok.com and get your authtoken
NGROK_AUTH_TOKEN = ""  # @param {type:"string"}

if NGROK_AUTH_TOKEN:
    !ngrok authtoken {NGROK_AUTH_TOKEN}
    print("✅ ngrok authenticated")
else:
    print("⚠️ ngrok not authenticated - public URL may be rate limited")

In [ ]:
import threading
import time
from pyngrok import ngrok

# Start uvicorn in background thread
def run_server():
    import uvicorn
    from retrofitkit.api.server import app
    uvicorn.run(app, host="0.0.0.0", port=8001, log_level="warning")

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

# Wait for server to start
time.sleep(5)

# Create ngrok tunnel
public_url = ngrok.connect(8001)

print("\n" + "="*60)
print("🚀 POLYMORPH-LITE Backend Running!")
print("="*60)
print(f"\n📡 Public URL: {public_url}")
print(f"📖 API Docs:   {public_url}/docs")
print(f"❤️  Health:     {public_url}/health")
print(f"📊 Metrics:    {public_url}/metrics")
print("\n" + "="*60)

## 6️⃣ Test the API

In [ ]:
import requests

# Test health endpoint
response = requests.get("http://localhost:8001/health")
print(f"Health check: {response.status_code}")
print(response.json())

In [ ]:
# Test metrics endpoint
response = requests.get("http://localhost:8001/metrics")
print(f"Metrics: {response.status_code}")
print(response.text[:500] + "...")

## 7️⃣ Run PMM (AI Brain) Demo

In [ ]:
# Initialize the PMM brain
sys.path.insert(0, '/content/POLYMORPH_LITE_MAIN/bentoml_service')

from pmm_brain import StaticPseudoModeMemory
import torch

# Create PMM instance
pmm = StaticPseudoModeMemory(latent_dim=128, max_modes=32, init_modes=4)
print(f"✅ PMM initialized with {pmm.n_active} active modes")

In [ ]:
# Simulate some spectra and process through PMM
import numpy as np

# Generate synthetic Raman-like latent vectors
np.random.seed(42)
num_samples = 50

for i in range(num_samples):
    # Simulate a latent vector (as if from an encoder)
    latent = torch.randn(1, 128)
    
    # Process through PMM
    recon, comp = pmm(latent)
    
    # Apply explicit updates periodically
    if i % 10 == 9:
        info = pmm.apply_explicit_updates()
        print(f"Step {i+1}: Active modes = {pmm.n_active}, Polymorphs = {pmm.poly_id}")

# Get final stats
stats = pmm.get_mode_stats()
print(f"\n📊 Final Stats:")
print(f"   Active modes: {stats['n_active']}/{stats['max_modes']}")
print(f"   Polymorphs discovered: {stats['poly_count']}")

In [ ]:
# Save PMM checkpoint
checkpoint_path = pmm.save_state('/content/POLYMORPH_LITE_MAIN/checkpoints', org_id='colab_demo')
print(f"✅ Checkpoint saved: {checkpoint_path}")

# List checkpoints
!ls -la /content/POLYMORPH_LITE_MAIN/checkpoints/

## 8️⃣ Test Gating Engine

In [ ]:
from retrofitkit.core.gating import GatingEngine

# Create gating engine with threshold rule
rules = [
    {
        "name": "peak_threshold",
        "direction": "above",
        "threshold": 100,
        "consecutive": 3,
    }
]

gating = GatingEngine(rules, cooldown_sec=5.0)

# Simulate spectra readings
readings = [50, 60, 110, 120, 130, 50, 140, 150, 160]  # Peak intensities

for t, intensity in enumerate(readings):
    triggered = gating.update({"t": t, "peak_intensity": intensity})
    status = "🔔 TRIGGERED" if triggered else "⏳ monitoring"
    print(f"t={t}: intensity={intensity:3d} → {status}")

print(f"\n📊 Gating Stats: {gating.get_stats()}")

## 9️⃣ Test Safety Guardrails

In [ ]:
from retrofitkit.core.safety.guardrails import SafetyGuardrails
import numpy as np

# Create safety guardrails
safety = SafetyGuardrails(
    max_intensity=65535,
    flatline_samples=5
)

# Test with normal spectrum
normal_spectrum = np.random.uniform(1000, 10000, 1024)
safe, msg = safety.check_over_intensity(normal_spectrum)
print(f"Normal spectrum: safe={safe}, msg='{msg}'")

# Test with over-intensity
over_spectrum = np.array([70000, 60000, 50000] + [1000] * 1021)
safe, msg = safety.check_over_intensity(over_spectrum)
print(f"Over-intensity:  safe={safe}, msg='{msg}'")

# Test with NaN values
nan_spectrum = np.array([np.nan] * 100 + [1000] * 924)
safe, msg = safety.check_invalid_spectrum(nan_spectrum)
print(f"NaN spectrum:    safe={safe}, msg='{msg}'")

print(f"\n📊 Safety Stats: {safety.get_stats()}")

## 🛑 Cleanup

In [ ]:
# Disconnect ngrok tunnel when done
ngrok.disconnect(public_url)
print("✅ ngrok tunnel closed")

---

## 📚 Next Steps

1. **Connect your frontend**: Use the ngrok public URL as your API endpoint
2. **Run more experiments**: Modify the PMM parameters and test different gating rules
3. **Save checkpoints**: Download checkpoints for use in production

---

*POLYMORPH-LITE by Dawson Block*